In [ ]:
# Install cirq and openfermion for use in google Colab
!pip install --quiet cirq;
!pip install --quiet openfermion;
import cirq
import openfermion
import matplotlib.pyplot as plt
import sympy
import numpy as np

###**Hamiltonian: XXZ in external field** 
The spin Hamiltonian used is the 1D antiferromagnetic XXZ chain with anisotropy parameter Δ and external field strength λ with periodic boundary conditions.

$$H(Δ, λ) = Σ_{i=1}^n σ^x_iσ^x_{i+1} + σ^y_iσ^y_{i+1} + Δσ^z_iσ^z_{i+1} + λΣ_{i=1}^n σ_i^z$$

In [ ]:
def XXZ_Ham(num_qubits, lam, delta):
  Ham = 0
  n = len(num_qubits)
  for i in range(n):
    Ham += cirq.X(num_qubits[i])*cirq.X(num_qubits[(i+1)%n])
    Ham += cirq.Y(num_qubits[i])*cirq.Y(num_qubits[(i+1)%n])
    Ham += delta*cirq.Z(num_qubits[i])*cirq.Z(num_qubits[(i+1)%n]) #with %n for periodic BCs 
    Ham += lam * cirq.Z(num_qubits[i])
  return(Ham)

In [ ]:
def exact(num_qubits, lam, delta):
  ham_uni = XXZ_Ham(num_qubits, lam, delta).matrix(num_qubits)
  energy = np.linalg.eigvals(ham_uni)
  return(min(energy))

In [ ]:
line_qubits = cirq.LineQubit.range(3)

print("The lowest eigenvalue =", exact(line_qubits,2,2))
print("The ground state energy =", exact(line_qubits,2,2).real)

The lowest eigenvalue = (-6+0j)
The ground state energy = -6.0


Generate the gates for the processing step given a vector of parameters $\vec{θ}$



In [ ]:
def rot_gate(params, qubit):
  yield cirq.rz(2*params[0]).on(qubit)
  yield cirq.ry(2*params[1]).on(qubit)

def processing_layer(params, qubits, length):
  n = int(len(qubits))
  for l in range(length):
    for i in range(n):
      yield rot_gate(params[l][i], qubits[i])
    for i in range(int(n/2)): #Append alternating CNOTs
      yield cirq.CNOT(qubits[(2*i+(l%2))%n], qubits[(2*i+1+(l%2))%n])

In [ ]:
line_qubits = cirq.LineQubit.range(4)
l = len(line_qubits)
length = 3

pro_params = np.full((length,l,2),0.5) #with 2*L params per layer, all 0.5 for testing


print(cirq.Circuit(processing_layer(pro_params, line_qubits, length)))

                                                              ┌──┐
0: ───Rz(0.318π)───Ry(0.318π)───@───Rz(0.318π)───Ry(0.318π)─────X────Rz(0.318π)───Ry(0.318π)───@───
                                │                               │                              │
1: ───Rz(0.318π)───Ry(0.318π)───X───Rz(0.318π)───Ry(0.318π)────@┼────Rz(0.318π)───Ry(0.318π)───X───
                                                               ││
2: ───Rz(0.318π)───Ry(0.318π)───@───Rz(0.318π)───Ry(0.318π)────X┼────Rz(0.318π)───Ry(0.318π)───@───
                                │                               │                              │
3: ───Rz(0.318π)───Ry(0.318π)───X───Rz(0.318π)───Ry(0.318π)─────@────Rz(0.318π)───Ry(0.318π)───X───
                                                              └──┘


Generate the gates for the encoding step given $Δ$, a vector of parameters $\vec{ϕ}$ and a function $f$

(Reusing the generator from the processing step)

In [ ]:
def rot_gate(params, qubit, delta):
  if delta == 0:
    yield cirq.rz(2*params[0]).on(qubit)
    yield cirq.ry(2*params[1]).on(qubit)
  else:
    yield cirq.rz(2*params[0]).on(qubit)
    yield cirq.ry(2*params[1]).on(qubit)
    yield cirq.rz(2*(params[0]+delta*params[2])).on(qubit)
    yield cirq.ry(2*(params[1]+delta*params[3])).on(qubit)

def processing_layer(params, qubits, length, delta):
  n = int(len(qubits))
  for l in range(length):
    for i in range(n):
      yield rot_gate(params[l][i], qubits[i], delta)
    for i in range(int(n/2)): #Append alternating CNOTs
      yield cirq.CNOT(qubits[(2*i+(l%2))%n], qubits[(2*i+1+(l%2))%n])

In [ ]:
line_qubits = cirq.LineQubit.range(4)
l = len(line_qubits)
length = 3

pro_params = np.full((length,l,4),0.5) #with 4*L params per layer, all 0.5 for testing


print(cirq.Circuit(processing_layer(pro_params, line_qubits, length, delta=0)))

                                                              ┌──┐
0: ───Rz(0.318π)───Ry(0.318π)───@───Rz(0.318π)───Ry(0.318π)─────X────Rz(0.318π)───Ry(0.318π)───@───
                                │                               │                              │
1: ───Rz(0.318π)───Ry(0.318π)───X───Rz(0.318π)───Ry(0.318π)────@┼────Rz(0.318π)───Ry(0.318π)───X───
                                                               ││
2: ───Rz(0.318π)───Ry(0.318π)───@───Rz(0.318π)───Ry(0.318π)────X┼────Rz(0.318π)───Ry(0.318π)───@───
                                │                               │                              │
3: ───Rz(0.318π)───Ry(0.318π)───X───Rz(0.318π)───Ry(0.318π)─────@────Rz(0.318π)───Ry(0.318π)───X───
                                                              └──┘


Combine these to write a function that returns the expectation value of
the Hamiltonian on the ansatz state.

In [ ]:
def cost_fun(final_state, num_qubits, lam, delta):
  return openfermion.expectation(XXZ_Ham(num_qubits, lam, delta).matrix(num_qubits)[0], final_state)

In [ ]:
simulator = cirq.Simulator()

circuit = cirq.Circuit(cirq.identity_each(*line_qubits))

final_state = simulator.simulate(circuit, initial_state=0).final_state_vector

print("The energy of the", cirq.dirac_notation(final_state), "state is", cost_fun(final_state, line_qubits, 1, 1))

The energy of the |0000⟩ state is (8+0j)


In [ ]:
def objective(num_qubits, lam, delta):
  circuit = 

AttributeError: ignored